# Feature Visualizations

## Required Packages

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from progressbar import ProgressBar, Bar, Percentage
from skimage.restoration import denoise_bilateral
from keras.optimizers import Adadelta, Adam
from skimage.transform import resize
from skimage.filters import gaussian
from imageio import imwrite, imread
from keras.layers import Lambda
from keras.layers import Input
from keras import regularizers
from keras import backend as K
from keras import Model

import tensorflow as tf
import numpy as np
import sys
import os

Using TensorFlow backend.


## Init

In [0]:
def make_directories():
    if not os.path.exists('docs'):
        os.mkdir('docs')

    if not os.path.exists('bin'):
        os.mkdir('bin')

    if not os.path.exists(os.path.join('bin', 'shapes')):
        os.mkdir(os.path.join('bin', 'shapes'))

In [0]:
make_directories()

## Utils

In [0]:
# referenced from https://github.com/tensorflow/lucid/blob/master/lucid/optvis/overrides/redirected_relu_grad.py#L90

@tf.RegisterGradient('RedirectedRelu')
def redirected_relu_grad(op, grad):
    x = op.inputs[0]
    relu_grad = tf.where(x < 0., tf.zeros_like(grad), grad)
    neg_pushing_lower = tf.logical_and(x < 0., grad > 0.)
    redirected_grad = tf.where(neg_pushing_lower, tf.zeros_like(grad), grad)
    assert_op = tf.Assert(tf.greater(tf.rank(relu_grad), 1), [tf.rank(relu_grad)])
    with tf.control_dependencies([assert_op]):
        batch = tf.shape(relu_grad)[0]
        reshaped_relu_grad = tf.reshape(relu_grad, [batch, -1])
        relu_grad_mag = tf.norm(reshaped_relu_grad, axis=1)
    result_grad = tf.where(relu_grad_mag > 0., relu_grad, redirected_grad)
    return result_grad

In [0]:
# referenced from https://github.com/tensorflow/lucid/blob/master/lucid/optvis/overrides/redirected_relu_grad.py#L117

@tf.RegisterGradient('RedirectedRelu6')
def redirected_relu6_grad(op, grad):
    x = op.inputs[0]
    relu6_cond = tf.logical_or(x < 0., x > 6.)
    relu_grad = tf.where(relu6_cond, tf.zeros_like(grad), grad)
    neg_pushing_lower = tf.logical_and(x < 0., grad > 0.)
    pos_pushing_higher = tf.logical_and(x > 6., grad < 0.)
    dir_filter = tf.logical_or(neg_pushing_lower, pos_pushing_higher)
    redirected_grad = tf.where(dir_filter, tf.zeros_like(grad), grad)
    assert_op = tf.Assert(tf.greater(tf.rank(relu_grad), 1), [tf.rank(relu_grad)])
    with tf.control_dependencies([assert_op]):
        batch = tf.shape(relu_grad)[0]
        reshaped_relu_grad = tf.reshape(relu_grad, [batch, -1])
        relu_grad_mag = tf.norm(reshaped_relu_grad, axis=1)
    result_grad = tf.where(relu_grad_mag > 0., relu_grad, redirected_grad)
    return result_grad

In [0]:
def get_model(model_name):
    model = None
    if model_name == 'mobile-net-v1':
        from keras.applications.mobilenet import MobileNet
        model = MobileNet(weights='imagenet', include_top=False,
                          input_shape=(None, None, 3))
    elif model_name == 'mobile-net-v2':
        from keras.applications.mobilenet_v2 import MobileNetV2
        model = MobileNetV2(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'vgg-16':
        from keras.applications.vgg16 import VGG16
        model = VGG16(weights='imagenet', include_top=False,
                      input_shape=(None, None, 3))
    elif model_name == 'vgg-19':
        from keras.applications.vgg19 import VGG19
        model = VGG19(weights='imagenet', include_top=False,
                      input_shape=(None, None, 3))
    elif model_name == 'densenet-121':
        from keras.applications.densenet import DenseNet121
        model = DenseNet121(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'densenet-169':
        from keras.applications.densenet import DenseNet169
        model = DenseNet169(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'densenet-201':
        from keras.applications.densenet import DenseNet201
        model = DenseNet201(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'resnet-50':
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(weights='imagenet', include_top=False,
                         input_shape=(None, None, 3))
    elif model_name == 'xception':
        from keras.applications.xception import Xception
        model = Xception(weights='imagenet', include_top=False,
                         input_shape=(None, None, 3))
    elif model_name == 'inception-v3':
        from keras.applications.inception_v3 import InceptionV3
        model = InceptionV3(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'inceptionresnet-v2':
        from keras.applications.inception_resnet_v2 import InceptionResNetV2
        model = InceptionResNetV2(weights='imagenet', include_top=False,
                                  input_shape=(None, None, 3))
    # for layer in model.layers:
    #     layer.trainable = False
    # model.compile('adadelta', 'mse')
    return model

In [0]:
def deprocess_image(x):
    return np.clip(((x - np.min(x)) / (np.max(x) - np.min(x))) * 255.0, 0, 255).astype('uint8')

In [0]:
def gaussian_blur(x, r):
    return gaussian(x, sigma=[0, r, r, 0], multichannel=True, preserve_range=True)

In [0]:
def denoise(x, color, spatial):
    x = denoise_bilateral(x, sigma_color=color, sigma_spatial=spatial,
                          mode='reflect', multichannel=True)
    return np.clip(x, 0, 1)

In [0]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [0]:
def logit(x):
    return np.log(x / (1 - x))

## Sample Generator

In [0]:
def main(model_name, layer_name, img_size):
    if not os.path.exists(os.path.join('bin', model_name)):
        os.mkdir(os.path.join('bin', model_name))
    if not os.path.exists(os.path.join('bin', os.path.join(model_name, layer_name.split('/')[0]))):
        os.mkdir(os.path.join('bin', os.path.join(model_name, layer_name.split('/')[0])))
        
    np.random.seed(10)

    base_model = None
    layer_dict = None
    model = None
    
    with K.get_session().graph.gradient_override_map({'Relu': 'RedirectedRelu', 'Relu6': 'RedirectedRelu6'}):
        base_model = get_model(model_name)
        layer_dict = dict([(layer.name, layer) for layer in base_model.layers])
        print(layer_dict)
        model = Model(inputs=layer_dict['input_1'].input, outputs=layer_dict[layer_name].output)
        datagen = ImageDataGenerator(rotation_range=30, fill_mode='reflect',
                                     zoom_range=0.1)
        regularization = regularizers.l2(0.001)
        optimization = Adam(lr=0.05)
        
        bar = ProgressBar(maxval=layer_dict[layer_name].output_shape[3],
                          widgets=[Bar('=', '[', ']'), ' ', Percentage()]).start()

        for i in range(layer_dict[layer_name].output_shape[3]):
            img_placeholder = K.placeholder(name='img_placeholder',
                                            shape=(2, img_size, img_size, 3))
            img_parameters = K.zeros(shape=(2, img_size, img_size, 3))
            img_parameters = K.update(img_parameters, img_placeholder)
            model_output = model(img_parameters)
            loss = K.mean(model_output[0, :, :, i]) - K.mean(model_output[1, :, :, i])
#             loss += regularization(img_parameters)
#             loss += 0.00001 * tf.reduce_sum(tf.image.total_variation(img_parameters))
            updates = optimization.get_updates(loss, [img_parameters])
            iterate = K.function(inputs=[img_placeholder, K.learning_phase()],
                                 outputs=[img_parameters, loss], updates=updates)

            radius = 0.25
            img = np.random.normal(size=(2, img_size, img_size, 3), loc=128.0, scale=0.01)
            filepath = os.path.join('bin', os.path.join(model_name,
                                                        layer_name.split('/')[0]))
            pos_name = os.path.join(filepath, 'pos-{:04d}.png'.format(i))
            neg_name = os.path.join(filepath, 'neg-{:04d}.png'.format(i))
            if os.path.exists(pos_name):
                pos_img = imread(pos_name)
                pos_img = (pos_img - pos_img.min()) / (pos_img.max() - pos_img.min())
                neg_img = imread(neg_name)
                neg_img = (neg_img - neg_img.min()) / (neg_img.max() - neg_img.min())

                if pos_img.shape[0] < img[0].shape[0]:
                    img[0] = resize(pos_img, img[0].shape[:2], preserve_range=True)
                    img[1] = resize(neg_img, img[1].shape[:2], preserve_range=True)
                    img = gaussian_blur(img, radius)
                else:
                    img[0] = pos_img
                    img[1] = neg_img

            best_pos = None
            best_neg = None
            minloss = float('inf')
            chance = 0
            cnt = 0
            while True:
                cnt += 1
                img, loss = iterate([img, 0])

                img = gaussian_blur(img, radius)

                if loss <= minloss:
                    minloss = loss
                    chance = 0
                    best_pos = img[0]
                    best_neg = img[1]
                else:
                    chance += 1

                if cnt % 512 == 0:
                    imwrite(pos_name, deprocess_image(best_pos))
                    imwrite(neg_name, deprocess_image(best_neg))

                if chance == 128:
                    break
            imwrite(pos_name, deprocess_image(best_pos))
            imwrite(neg_name, deprocess_image(best_neg))
            bar.update(i + 1)
    bar.finish()

In [13]:
main('mobile-net-v1', 'conv1', 128)

W0711 18:43:26.093011 140390420174720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0711 18:43:26.094771 140390420174720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0711 18:43:26.096078 140390420174720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0711 18:43:26.209385 140390420174720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

/usr/local/lib/python3.6/dist-packages

17227776/17225924 [==============================] - 1s 0us/step


[                                                                        ] N/A%

{'input_1': <keras.engine.input_layer.InputLayer object at 0x7faef8f13860>, 'conv1_pad': <keras.layers.convolutional.ZeroPadding2D object at 0x7faeee4c0fd0>, 'conv1': <keras.layers.convolutional.Conv2D object at 0x7faeee4c0908>, 'conv1_bn': <keras.layers.normalization.BatchNormalization object at 0x7faf0f3d24a8>, 'conv1_relu': <keras.layers.advanced_activations.ReLU object at 0x7faf0f431be0>, 'conv_dw_1': <keras.layers.convolutional.DepthwiseConv2D object at 0x7faeee4db1d0>, 'conv_dw_1_bn': <keras.layers.normalization.BatchNormalization object at 0x7faee8cc3828>, 'conv_dw_1_relu': <keras.layers.advanced_activations.ReLU object at 0x7faee8c92fd0>, 'conv_pw_1': <keras.layers.convolutional.Conv2D object at 0x7faee8c42f98>, 'conv_pw_1_bn': <keras.layers.normalization.BatchNormalization object at 0x7faee8bc0160>, 'conv_pw_1_relu': <keras.layers.advanced_activations.ReLU object at 0x7faee8bc0f60>, 'conv_pad_2': <keras.layers.convolutional.ZeroPadding2D object at 0x7faee8b3fa20>, 'conv_dw_2':

[==                                                                      ]   3%

KeyboardInterrupt: ignored

In [0]:
#!zip -r inception-v3-2 bin/inception-v3/conv2d_2

In [0]:
#!rm -rf bin